In [7]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
# import seaborn as sns
# from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import math
from sklearn.metrics import accuracy_score
rng = np.random.default_rng(51)

import scipy.integrate as spi


In [2]:
header = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv('cars/car.data' ,names=header)

#header = ['class', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']
#df = pd.read_csv('letter_recognition/letter-recognition.data', names=header)

df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [3]:
le = preprocessing.LabelEncoder()
original_classes = np.unique(df['class'].values)
for column_name in df.columns:
    df[column_name] = le.fit_transform(df[column_name])
y = df['class']
X = df.loc[:, df.columns != 'class']

In [4]:
X = X.values
Y = y.values.astype(int)
feature_num = X.shape[1]
classes_num = len(pd.unique(y))
print('Numb of features: ', feature_num)
print('Numb of classes: ', classes_num)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(x_train.shape)
print(y_test.shape)

Numb of features:  6
Numb of classes:  4
(1382, 6)
(346,)


In [69]:
def gaussian_kernel(u):
    return np.exp(-0.5 * u**2) / ((2 * np.pi))

def epanechnikov_kernel(u):
    return 3/4 * (1 - u**2)

In [121]:
# 1. Выбор h
# 2. Выбрать функцию расстояния
def def_dist(x_i: np.ndarray, X: np.ndarray) -> np.ndarray:
    return X - x_i

class ParzenRozenblatt:
    def __init__(self, kernel_func=gaussian_kernel, dist_func=def_dist) -> None:
        self.kernel = kernel_func
        self.dist_func = dist_func

    def fit(self, x_train, y_train, h):
        self.x = x_train
        self.y = y_train
        self.h = h

    def _get_V_h(self, distances):
        from scipy.integrate import quad as integrate
        a = np.max(distances)
        b = np.min(distances)
        V_h, _ = integrate(self.kernel, b, a)
        return V_h

    def predict_single_row(self, x_row, h):
        # Общая формула:
        # a = argmax[lam_y / V(h) * sum([if y_i == y] * self.kernel(dist(row, x_i)))]
        
        scores = np.zeros(np.unique(self.y).shape[0])
        distances_ = self.dist_func(x_row, self.x)

        # Нормировочный множитель
        V_h = self._get_V_h(distances_)

        for y_i in set(self.y):
            # p(x|y)
            indexes = np.where(self.y == y_i)[0]
            distances_tmp = distances_[indexes]
            distances = np.linalg.norm(distances_tmp, axis=1)
            weights = self.kernel(u=distances/h)
            scores[y_i] = sum(weights)
        
        prediction = np.argmax(scores/(len(self.x)*V_h), axis=0)
        return prediction
    
    def predict(self, x, h_candidates):
        pass
        # Надо сделать выбор лучшего h
        h = h_candidates[0]

        predictions = np.array([])
        
        for row in x:
            prediction = self.predict_single_row(row, h)
            predictions = np.append(predictions, prediction)
        return predictions
                

            

In [122]:
classifier = ParzenRozenblatt(kernel_func=epanechnikov_kernel)
classifier.fit(x_train, y_train, [0.1])

preds = classifier.predict(x_test, [0.1])

-4.999999999999999
-4.999999999999999
-1.0
-9.0
-9.0
-9.0
-1.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-9.0
-4.999999999999999
-9.0
-1.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-9.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-9.0
-4.999999999999999
-9.0
-4.999999999999999
-1.0
-9.0
-1.0
-1.0
-4.999999999999999
-4.999999999999999
-9.0
-4.0
-4.999999999999999
-4.999999999999999
-4.0
-4.999999999999999
-4.999999999999999
-1.0
-9.0
-4.999999999999999
-4.999999999999999
-1.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-9.0
-9.0
-4.999999999999999
-4.999999999999999
-1.0
-4.999999999999999
-9.0
-9.0
-9.0
-4.999999999999999
-1.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-9.0
-9.0
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.999999999999999
-4.99999999999

In [123]:
accuracy = accuracy_score(y_test, preds)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7254335260115607


In [107]:
def _get_V_h(distances):
    from scipy.integrate import quad as integrate
    # for j in range(distances.shape[1]):
    max = np.max(distances)
    min = np.min(distances)
    integral, _ = integrate(gaussian_kernel, min, max)
    return integral
    

In [108]:
row = x_test[0]
distances = x_train - row
_get_V_h(distances)

0.3893277594835447